In [1]:
import random
import pandas as pd

random.seed(123) # Seed for reproducreproducibility

c = 20 # Capacity of the knapsack

# Function to generate random values and weights
def generate_data():
    value = random.uniform(1, 100)
    weight = random.uniform(1, 10)
    return value, weight

# Generate data for 1000 observations
data = [{'Value': value, 'Weight': weight} for value, weight in (generate_data() for _ in range(1000))]

# Sort data based on value/weight ratio
data.sort(reverse = True, key=lambda x: x['Value'] / x['Weight'])

df = pd.DataFrame(data)
v = list(df['Value']) #list of values
w = list(df['Weight']) #list of weights 

N = len(w) # Number of items
N_list = list(range(N)) # list with item indexes

# Task 1
Primal Heuristic solver: Greedy algorithm

In [2]:
#Greedy knapsack algorithm (assuming knapsack items are sorted by density)

selected_items = []
total_weight = []
total_profit = []

def greedy_knapsack(weights, values, capacity, variables_set: list):
    #STEP 1. Fill knapsack with all items taken in the given sorted by densisty sequence
    selected_items = []
    total_weight = []
    total_profit = []
    
    sorted_indices = sorted(variables_set, key=lambda i: values[i] / weights[i], reverse=True)

    for i in sorted_indices: 
        remaining_weight = capacity - sum(total_weight)
        if weights[i] <= remaining_weight:
            total_weight.append(weights[i])
            selected_items.append(i)
            total_profit.append(values[i])   

    # STEP 2. Return a combination of items inside our knapsack
    return selected_items, remaining_weight, total_profit

selected_items, total_weight, total_profit = greedy_knapsack(w, v, c, N_list)

print(f'Items in the knapsack: {selected_items}')
print('Remaining capacity in the knapsack: %.2f' % total_weight)
print('Solution found. Objective function Value: %.2f' % sum(total_profit))

Items in the knapsack: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
Remaining capacity in the knapsack: 0.86
Solution found. Objective function Value: 1307.68


# Task 6

Implement the Greedy algorithm from exercise 1 for branch and cut. It will separate cuts. 

In [3]:
# 1 Import gurobi package

from gurobipy import *

In [4]:
# 2 Define an optimization model

knapsack_model = Model('01-knapsack')

Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-03


In [5]:
# 3 add decision variables (binary x {not take, take})

x = knapsack_model.addVars(N, vtype=GRB.BINARY, name="x")

In [6]:
# 4 define the objective function

obj_fn = sum(v[i]*x[i] for i in N_list)

knapsack_model.setObjective(obj_fn, GRB.MAXIMIZE)

In [7]:
# 5 add the constraints

knapsack_model.addConstr(sum(w[i]*x[i] for i in N_list) <= c)
knapsack_model.setParam(GRB.Param.LazyConstraints, 1)

knapsack_model.Params.Cuts = 0
knapsack_model.Params.Heuristics = 0
knapsack_model.Params.Presolve = 0

Set parameter LazyConstraints to value 1
Set parameter Cuts to value 0
Set parameter Heuristics to value 0
Set parameter Presolve to value 0


In [8]:
def callback(model, where):
        if where not in (GRB.Callback.MIPSOL, GRB.Callback.MIPNODE):
            return
        
        # In MIPNODE, we must ensure that we have the optimal solution to the
        # linear relaxation at that node. Otherwise we are not "authorised"
        # to call cbGetNodeRel. See:
        # https://www.gurobi.com/documentation/9.1/refman/py_model_cbgetnoderel.html
    
        if where == GRB.Callback.MIPNODE and model.cbGet(GRB.Callback.MIPNODE_STATUS) != GRB.OPTIMAL:
            return
        #If gurobi raises an error, then there's a fractional solution in that node
        try:
            xstar = model.cbGetSolution(x)
        except GurobiError:
            xstar = model.cbGetNodeRel(x)
              
        xstar_list = [i for i, x in xstar.items() if x != 0] #Create a list with the selected items
        cover = greedy_knapsack(w, v, c, xstar_list)[0]

        #separate violated cover inequalities, i.e., cutting planes to tighten the linear programming relaxations
        if len(cover) < len(xstar_list):
            #print(f"Violated cover inequality. Cover = {len(cover):.3f} < {len(xstar_list):.3f}") deactivated because it prints a lot of them
            cut = sum(x[i] for i in xstar_list) <= (len(cover) + 1e-6)
            model.cbLazy(cut)
    



In [9]:
# 6 solve the model and output the solution

knapsack_model.optimize(callback)

print('Optimization completed. Objective function Value: %.2f' % knapsack_model.objVal)

#write a loop to print each variable's solution given by the solver

#for v in knapsack_model.getVars():

    #print(f"{v.VarName} = {v.x}")

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (linux64)

CPU model: 12th Gen Intel(R) Core(TM) i7-12700H, instruction set [SSE2|AVX|AVX2]
Thread count: 20 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 1 rows, 1000 columns and 1000 nonzeros
Model fingerprint: 0xfc90aed5
Variable types: 0 continuous, 1000 integer (1000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [1e+00, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+01, 2e+01]
Variable types: 0 continuous, 1000 integer (1000 binary)

Root relaxation: objective 1.360597e+03, 1 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1360.59669    0    1          - 1360.59669      -     -    0s
Violated cover inequality. Cover = 15.000 < 16.000
     0     0 1360.59669    0    1          - 1360